## Benchmarking

В этом блокноте мы рассмотрим важность «интерполяции» глобальных переменных при сравнении функций. Мы интерполируем глобальную переменную, бросая перед ней **$**. 

```julia
@benchmark sum($A)
```

а не

```julia
@benchmark sum(A)
```

Давайте посмотрим, может ли это изменить ситуацию, проверив соотношение во времени выполнения `sum ($A)` и `sum (A)` для массивов разного размера `A`.

#### Упражнение

Вызовите функцию «sum» для псевдослучайно заполненного одномерного массива, который называется «foo», и имеет длину от $2$ до $2 ^ {20} (~ 10 ^ 6)$. Для каждого размера `foo` определите соотношение времени выполнения для `sum (foo)` и `sum ($foo)`. (Чтобы определить это соотношение, используйте минимальное время выполнения в каждом случае.) Постройте соотношение времени выполнения для неинтерполированных и интерполированных `foo` в вызовах `sum` и длины `foo`. Кажется ли, что интерполяция `foo` имеет значение? Если да, то для каких размеров `foo`?

## Производительность -- стабильность типов

Одним из способов оптимизации кода в Julia является обеспечение **стабильности типов**. Если тип(ы) некоторых переменных в функции подвержен изменениям или неоднозначности, компилятор также не может рассуждать об этих переменных, и производительность будет падать. И наоборот, мы разрешаем компилятору оптимизировать и генерировать более эффективный машинный код, когда мы объявляем переменные, чтобы их типы были фиксированы по всему телу функции.

Например, допустим, у нас были функции с именами `baz` и` bar` со следующими определениями

```julia
function baz()
    s = rand()
    if s > 2/3
        return .666667
    elseif s > 1/3
        return 1//3    
    else
        return 0    
    end
end
```

```julia
function bar()
    s = rand()
    if s > 2/3
        return .666667
    elseif s > 1/3
        return .3333333    
    else
        return 0.0    
    end
end
```

Когда я сравниваю их через

```julia
using BenchmarkTools
@benchmark baz()
@benchmark bar()
```

Я вижу, что `bar` почти в три раза быстрее, чем` baz`! Причина в том, что `bar` является стабильным по типу, а` baz` - нет: компилятор может сказать, что `bar` всегда будет возвращать` Float`, тогда как `baz` может возвращать` Float`, `Int` или `Rational`. Когда компилятор может определить тип выходных данных функции или переменных, объявленных *внутри функции*, без выполнения кода, он может работать намного лучше.

#### Упражнение

Следующее определение для `my_sum` не можнт отличиться стабильным типом.

```julia
function my_sum(A)
    output = 0
    for x in A
        output += x
    end
    return output
end
```

Скопируйте и выполните приведенный выше код в новую ячейку. Оцените его, используя `A = rand (10 ^ 3)`. Затем напишите новую функцию my_sum2 с тем же телом функции, что и my_sum. Обновите `my_sum2`, чтобы сделать его стабильным, и сравните его для произвольно заполненного массива с 10 ^ 3 записями.

Насколько стабильность типа влияет на производительность? Если вы хотите, попробуйте это же упражнение для нескольких размеров `A`, чтобы увидеть, изменит ли это ваш ответ!

#### Упражнение

Сделайте следующий тип кода стабильным. Вы поймете, что ваши усилия окупаются, когда увидите повышение производительности!

```julia
"""
    my_sqrt(x)
    
Вычислить квадратный корень из `x` методом Ньютона. # ???А не вавилоский ли
"""
function my_sqrt(x)
    output = 1
    for i in 1:1000
        output = .5 * (output + x/output)
    end
    output
end
```